<a href="https://colab.research.google.com/github/PravinTiwari023/SparkNLP-Project/blob/main/sparknlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spark-nlp==5.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=edb657d96cc00a312c561f19d9e6b559adb8a503a83b6a23aea9bac9a31b3f1d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Ignore this part
import pandas as pd
import numpy as np
import json

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [ ]:
# Ignore this part
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 5.2.2


In [ ]:
# Ignore this part
document_assembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("documents")

t5_ = T5Transformer() \
  .pretrained("t5_small", 'en') \
  .setTask("summarize:")\
  .setMaxOutputLength(200)\
  .setInputCols(["documents"]) \
  .setOutputCol("summaries")

summarizer_pp = Pipeline(stages=[document_assembler, t5_])

t5_small download started this may take some time.
Approximate size to download 241.9 MB
[OK!]


In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

t5 = T5Transformer.pretrained("t5_passive_to_active_styletransfer") \
    .setTask("transfer Passive to Active:") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(200) \
    .setOutputCol("transfers")

pipeline = Pipeline().setStages([documentAssembler, t5])

t5_passive_to_active_styletransfer download started this may take some time.
Approximate size to download 253.2 MB
[OK!]


In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

## If you are running for first time
# import json
# d={}
# with open('/content/drive/MyDrive/Database/users.json','w') as f:
#   json.dump(d,f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from typing_extensions import Text
import json
class NLPApp:
  # Constructor
  def __init__(self):
    self.__First_menu()

  def __First_menu(self):
    first_input = input('''Hi! How would you like to proceed?
    1. Register
    2. Login
    3. Exit\n''')

    if first_input == '1':
      # SignUp
      self.__Register()
      pass

    elif first_input == '2':
      # SignIn
      self.__Login()
      pass

    else:
      # Exit
      print('Thank You')
      exit()

  def __Second_menu(self):
    print('\nWelcome')
    first_input = input('''Hi! How would you like to proceed?
    1. Text Summarization
    2. Passive to Active
    3. Exit\n''')

    if first_input == '1':
      # SignUp
      self.__Text_summary()
      pass

    elif first_input == '2':
      # SignIn
      self.__Active()
      pass

    else:
      # Exit
      print('Thank You')
      exit()

  def __Register(self):
    name = input('Name: ')
    email = input('Email: ')
    password = input('Password: ')
    cpassword = input('Confirm Password: ')

    if cpassword == password:
      self.insert(name, email, password)
    else:
      print('\nPassword are not matching')
      self.__Register()

  def __Login(self):
    email = input('Email: ')
    password = input('Password: ')

    self.search(email, password)

  def __Text_summary(self):
    text = input('Enter Text: ')
    empty_df = spark.createDataFrame([['']]).toDF('text')
    pipeline_model = summarizer_pp.fit(empty_df)
    sum_lmodel = LightPipeline(pipeline_model)
    res = sum_lmodel.fullAnnotate(text)[0]
    print('Summary:', res['summaries'][0].result)
    self.__Second_menu()

  def __Active(self):
    text = input('Enter Text: ')
    pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF('text'))
    T5model = LightPipeline(pipeline_model)
    res = T5model.fullAnnotate(text)[0]
    print ('Prediction:', res['transfers'][0].result)
    self.__Second_menu()

  def insert(self,name,email,password):
    with open('/content/drive/MyDrive/Database/users.json','r') as rf:
        users = json.load(rf)

        if email in users:
            print('Email is already registered')
            self.__First_menu()
        else:
            users[email] = [name,password]

    with open('/content/drive/MyDrive/Database/users.json','w') as wf:
        json.dump(users,wf,indent=4)
        print('Registered Successfully. Welcome!!')
        self.__Login()

  def search(self,email,password):
    with open('/content/drive/MyDrive/Database/users.json','r') as rf:
        users = json.load(rf)

        if email in users:
            if users[email][1] == password:
                self.__Second_menu()
            else:
                print('Incorrect password!!')
                self.__Second_menu()
        else:
          print('Email is not registered yet!!')
          self.__First_menu()


# Creating class object
App = NLPApp()

Hi! How would you like to proceed?
    1. Register
    2. Login
    3. Exit
2
Email: tiwaripravin114@gmail.com
Password: Pravin@007

Welcome
Hi! How would you like to proceed?
    1. Text Summarization
    2. Passive to Active
    3. Exit
3
Thank You
